# Quantization using the Model Compression Toolkit - example in Pytorch
[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/mct_features_notebooks/pytorch/example_pytorch_ptq_mnist.ipynb)

## Overview
This quick-start guide explains how to use the Model Compression Toolkit (MCT) to quantize a PyTorch model. We'll provide an end-to-end example, starting with training a model from scratch on MNIST dataset and then applying MCT for quantization.

## Summary
In this tutorial, we will explore the following:

**1. Training a PyTorch model from scratch on MNIST:** We'll start by building a basic PyTorch model and training it on the MNIST dataset.
**2. Quantizing the model using 8-bit activations and weights:** We'll employ a hardware-friendly quantization technique, such as symmetric quantization with power-of-2 thresholds.
**3. Evaluating the quantized model:** We'll compare the performance of the quantized model to the original model, focusing on accuracy.
**4. Analyzing compression gains:** We'll estimate the compression achieved by quantization.

## Setup
Install the relevant packages:

In [ ]:
! pip install -q torch
! pip install -q torchvision
! pip install -q onnx

In [ ]:
import importlib
if not importlib.util.find_spec('model_compression_toolkit'):
    !pip install model_compression_toolkit

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

## Train a Pytorch classifier model on MNIST
Let's define the network and a few helper functions to train and evaluate the model. The following code snippets are adapted from the official PyTorch examples: https://github.com/pytorch/examples/blob/main/mnist/main.py

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

batch_size = 64
test_batch_size = 1000
random_seed = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)
epochs = 2
gamma = 0.7
lr = 1.0

Let's define the dataset loaders and optimizer, then train the model for 2 epochs.

In [ ]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset_folder = './mnist'
train_dataset = datasets.MNIST(dataset_folder, train=True, download=True,
                   transform=transform)
test_dataset = datasets.MNIST(dataset_folder, train=False,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, num_workers=1, pin_memory=True, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, num_workers=1, pin_memory=True,  batch_size=test_batch_size, shuffle=False)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

## Representative Dataset
For quantization with MCT, we need to define a representative dataset required by the Post-Training Quantization (PTQ) algorithm. This dataset is a generator that returns a list of images:

In [ ]:
n_iter=10

def representative_dataset_gen():
    dataloader_iter = iter(train_loader)
    for _ in range(n_iter):
        yield [next(dataloader_iter)[0]]

## Hardware-friendly quantization using MCT
Now for the exciting part! Let’s run hardware-friendly post-training quantization on the model. 

In [ ]:
import model_compression_toolkit as mct

# Define a `TargetPlatformCapability` object, representing the HW specifications on which we wish to eventually deploy our quantized model.
target_platform_cap = mct.get_target_platform_capabilities('pytorch', 'default')

quantized_model, quantization_info = mct.ptq.pytorch_post_training_quantization(
        in_module=model,
        representative_data_gen=representative_dataset_gen,
        target_platform_capabilities=target_platform_cap
)

Our model is now quantized. MCT has created a simulated quantized model within the original PyTorch framework by inserting [quantization representation modules](https://github.com/sony/mct_quantizers). These modules, such as `PytorchQuantizationWrapper` and `PytorchActivationQuantizationHolder`, wrap PyTorch layers to simulate the quantization of weights and activations, respectively. While the size of the saved model remains unchanged, all the quantization parameters are stored within these modules and are ready for deployment on the target hardware. In this example, we used the default MCT settings, which compressed the model from 32 bits to 8 bits, resulting in a compression ratio of 4x. Let's print the quantized model and examine the quantization modules:

In [ ]:
print(quantized_model)

## Models evaluation
Using the simulated quantized model, we can evaluate its performance and compare the results to the floating-point model.

Let's start with the floating-point model evaluation.

In [ ]:
test(model, device, test_loader)

Finally, let's evaluate the quantized model:

In [ ]:
test(quantized_model, device, test_loader)

Now, we can export the quantized model to ONNX:

In [ ]:
mct.exporter.pytorch_export_model(quantized_model, save_model_path='qmodel.onnx', repr_dataset=representative_dataset_gen)

## Conclusion

In this tutorial, we demonstrated how to quantize a classification model for MNIST in a hardware-friendly manner using MCT. We observed that a 4x compression ratio was achieved with minimal performance degradation.

The key advantage of hardware-friendly quantization is that the model can run more efficiently in terms of runtime, power consumption, and memory usage on designated hardware.

While this was a simple model and task, MCT can deliver competitive results across a wide range of tasks and network architectures. For more details, [check out the paper:](https://arxiv.org/abs/2109.09113).


Copyright 2022 Sony Semiconductor Israel, Inc. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
